In [5]:
import pandas as pd
import numpy as np
import datetime
from sklearn import preprocessing
from tqdm import tqdm
tqdm.pandas(desc="")

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
dt = dt = pd.read_excel('../data/primary.xlsx')
dt.to_csv('../data/primary.csv')
# dt = pd.read_csv('../data/primary.csv')

In [7]:
# dt[['DRV_KBM', 'DRV_YEAR']].head()
dt.head()

,DRV_KBM,DRV_YEAR
0,NaN,NaN
1,"0.85,0.85,0.85","51,29,49"
2,"0.8,0.8,0.8,0.8,0.7","52,30,50,49,43"
3,"0.8,0.8,0.8,0.8","52,30,50,49"
4,"1.,0.8,0.8,0.8","23,52,30,50"


In [8]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107081 entries, 0 to 107080
Data columns (total 55 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         107081 non-null  int64  
 1   AGR_ISN            107081 non-null  int64  
 2   AGR_DATESIGN       107081 non-null  object 
 3   AGR_DATEBEG        107081 non-null  object 
 4   AGR_DATEEND        107081 non-null  object 
 5   AGR_SRC            107081 non-null  object 
 6   AGR_CREATEDBY      107081 non-null  object 
 7   INS_ISN            107081 non-null  int64  
 8   INS_NAME           107081 non-null  object 
 9   INS_SEX            86784 non-null   object 
 10  INS_YEAR           86933 non-null   float64
 11  INS_JUR            107081 non-null  int64  
 12  INS_REG            104643 non-null  float64
 13  INS_DOCSER         86976 non-null   object 
 14  OWN_ISN            107081 non-null  int64  
 15  OWN_NAME           107081 non-null  object 
 16  OW

# План работ
Построим модель, предсказывающую вероятность убытка по параметрам договора

+ из дат подписания и начала можно получить информацию о том, насколько заранее куплен полис
+ из источника продаж, создателя, типа ТС, категории, марки, и популярных моделей получим среднюю убыточность по каждому из них
+ из года выпуска получим возраст машины на момент страхования
+ по каждому региону (собственника, страхователя, + по их паспортам) также получим среднюю убыточность
+ данных о регионах очень мало, но они очень важны. Попробуем заполнить недостающие данные о регионе использования ТС с помощью региона выдачи паспорта. Потом получим среднюю убыточность по региону
+ КБМ по договорам с юрлицами отсутствует, заполним его средними значениями КБМ по другим договорам
+ выведем отдельно признак пролонгируемого договора
+ пол закодируем в onehot, где он неизвестен (премущественно это мультидрайв) будет 0,0
+ отдельными колонками выведем общее количество водителей, минимальный и максимальный возраст, минимальный и максимальный КБМ

In [9]:
# из дат подписания и начала можно получить информацию о том, насколько заранее куплен полис

def daysbetween(d1, d2):
    dt1 = datetime.datetime.strptime(d1, '%d.%m.%y')
    dt2 = datetime.datetime.strptime(d2, '%d.%m.%y')
    return (dt2 - dt1).days

In [10]:
dt['f1_daysbeforestart'] = dt.apply(lambda x: daysbetween(x['AGR_DATESIGN'], x['AGR_DATEBEG']), axis=1)

In [11]:
# из источника продаж, создателя, типа ТС, категории, марки, и популярных моделей получим 
# нормализованную среднюю убыточность по каждому из них

class claimmeansclass():
    '''
    для инициации класса необходимо передать колонку, по которой считаем убытки и тип убытка 
    (по оплаченным убыткам если pay=True и по неоплаченным если False)
    '''
    def __init__(self, col, pay):
        min_max_scaler = preprocessing.MinMaxScaler()
        if pay == True:
            claim = 'CNT_CLAIM_PAY'
        elif pay == False:
            claim = 'CNT_CLAIM_NOT_PAY'
        mn = dt[[col, claim]].groupby(col).mean()
        sm = dt[[col, claim]].groupby(col).sum()
        mn[claim] = min_max_scaler.fit_transform(mn[[claim]])
        self.mnsm = mn.join(sm, on=col, rsuffix = '_SUM')
        self.claim = claim
#         print(self.mnsm)


    def claimmeans(self, val, limit = 0):
        '''Функция отдаёт нормированную среднюю убыточность по колонке col для ее значения val из датафрейма df. Отдаёт среднюю убыточность по колонке col,
        если убытков для анализа меньше, чем limit'''
    #     print(mnsm)
        if self.mnsm.loc[val][1] >= limit:
            return self.mnsm.loc[val, self.claim]
        else: return self.mnsm[self.claim].mean()

In [12]:
ctype = [True, False]
ftype = ['AGR_SRC', 'AGR_CREATEDBY', 'TS_TYPE', 'TS_CATEGORY', 'TS_MARK_NAME', 'TS_MODEL_NAME']

for index, f in enumerate(ftype):
    i = index+2
    dt[f].fillna('Отсутствует', inplace=True)
    for c in ctype:
        name = 'f' + str(i) + '_' + f + '_' + str(c)
        cmc = claimmeansclass(f, c)
        print(f, c, name)
        dt[name] = dt.progress_apply(lambda x: cmc.claimmeans(x[f], limit=10), axis = 1)
        

  0%|                                                                               | 5/107081 [00:00<36:01, 49.53it/s]

AGR_SRC True f2_AGR_SRC_True


  0%|                                                                             | 32/107081 [00:00<06:26, 276.98it/s]

AGR_SRC False f2_AGR_SRC_False


  1%|▌                                                                          | 838/107081 [00:00<2:04:05, 14.27it/s]

...

  0%|                                                                             | 1/107081 [00:00<3:37:34,  8.20it/s]

TS_MODEL_NAME True f7_TS_MODEL_NAME_True


  0%|                                                                             | 1/107081 [00:00<3:43:48,  7.97it/s]

TS_MODEL_NAME False f7_TS_MODEL_NAME_False


100%|████████████████████████████████████████████████████████████████████████| 107081/107081 [00:13<00:00, 7930.92it/s]


In [13]:
# после каждого шага сохраняем датасет. Крайняя фича - f7
dt.to_csv('../data/step1.csv')

In [95]:
pd.read_csv('../data/step1.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107081 entries, 0 to 107080
Data columns (total 69 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              107081 non-null  int64  
 1   Unnamed: 0.1            107081 non-null  int64  
 2   AGR_ISN                 107081 non-null  int64  
 3   AGR_DATESIGN            107081 non-null  object 
 4   AGR_DATEBEG             107081 non-null  object 
 5   AGR_DATEEND             107081 non-null  object 
 6   AGR_SRC                 107081 non-null  object 
 7   AGR_CREATEDBY           107081 non-null  object 
 8   INS_ISN                 107081 non-null  int64  
 9   INS_NAME                107081 non-null  object 
 10  INS_SEX                 86784 non-null   object 
 11  INS_YEAR                86933 non-null   float64
 12  INS_JUR                 107081 non-null  int64  
 13  INS_REG                 104643 non-null  float64
 14  INS_DOCSER          

In [15]:
# из года выпуска получим возраст машины на момент страхования


dt['f8_car_age_on_start'] = dt.apply(lambda x: datetime.datetime.strptime(x['AGR_DATEBEG'], '%d.%m.%y').year - x['TS_YEAR'], axis=1)

In [16]:
# По каждому региону (собственника, страхователя, + по их паспортам) также получим среднюю убыточность
dt['INS_DOCSER'] = dt['INS_DOCSER'].apply(lambda x: str(x)[:2])
dt['OWN_DOCSER'] = dt['OWN_DOCSER'].apply(lambda x: str(x)[:2])

In [17]:
dt['INS_DOCSER'].fillna(0, inplace=True)
dt['OWN_DOCSER'].fillna(0, inplace=True)

In [18]:
def intdocser(docser):
    try:
        return int(docser)
    except:
        return 0

In [19]:
dt['INS_DOCSER'] = dt['INS_DOCSER'].apply(intdocser)
dt['OWN_DOCSER'] = dt['OWN_DOCSER'].apply(intdocser)

In [20]:
dt.groupby('OWN_DOCSER').count().sort_values(by=['AGR_ISN'])

,Unnamed: 0,AGR_ISN,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_ISN,INS_NAME,INS_SEX,...,f3_AGR_CREATEDBY_False,f4_TS_TYPE_True,f4_TS_TYPE_False,f5_TS_CATEGORY_True,f5_TS_CATEGORY_False,f6_TS_MARK_NAME_True,f6_TS_MARK_NAME_False,f7_TS_MODEL_NAME_True,f7_TS_MODEL_NAME_False,f8_car_age_on_start
OWN_DOCSER,,,,,,,,,,,,,,,,,,,,,
21,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
16,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
13,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
6,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,4,4,4,4,4,4,4,4,4,3,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,3588,3588,3588,3588,3588,3588,3588,3588,3588,3581,...,3588,3588,3588,3588,3588,3588,3588,3588,3588,3588
46,8109,8109,8109,8109,8109,8109,8109,8109,8109,8104,...,8109,8109,8109,8109,8109,8109,8109,8109,8109,8109
60,8484,8484,8484,8484,8484,8484,8484,8484,8484,8476,...,8484,8484,8484,8484,8484,8484,8484,8484,8484,8484


In [21]:
# создадим регион 0 для тех, у кого он не определен
dt['OWN_REG'].fillna(0, inplace=True)
dt['INS_REG'].fillna(0, inplace=True)
dt.groupby('OWN_REG').count()

,Unnamed: 0,AGR_ISN,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_ISN,INS_NAME,INS_SEX,...,f3_AGR_CREATEDBY_False,f4_TS_TYPE_True,f4_TS_TYPE_False,f5_TS_CATEGORY_True,f5_TS_CATEGORY_False,f6_TS_MARK_NAME_True,f6_TS_MARK_NAME_False,f7_TS_MODEL_NAME_True,f7_TS_MODEL_NAME_False,f8_car_age_on_start
OWN_REG,,,,,,,,,,,,,,,,,,,,,
0.0,2506,2506,2506,2506,2506,2506,2506,2506,2506,2006,...,2506,2506,2506,2506,2506,2506,2506,2506,2506,2506
1.0,179,179,179,179,179,179,179,179,179,177,...,179,179,179,179,179,179,179,179,179,179
3.0,3731,3731,3731,3731,3731,3731,3731,3731,3731,3415,...,3731,3731,3731,3731,3731,3731,3731,3731,3731,3731
4.0,909,909,909,909,909,909,909,909,909,897,...,909,909,909,909,909,909,909,909,909,909
5.0,3331,3331,3331,3331,3331,3331,3331,3331,3331,2824,...,3331,3331,3331,3331,3331,3331,3331,3331,3331,3330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.0,41,41,41,41,41,41,41,41,41,39,...,41,41,41,41,41,41,41,41,41,41
96.0,1092,1092,1092,1092,1092,1092,1092,1092,1092,1054,...,1092,1092,1092,1092,1092,1092,1092,1092,1092,1092
97.0,103,103,103,103,103,103,103,103,103,101,...,103,103,103,103,103,103,103,103,103,103


In [22]:
rtype = ['OWN_REG', 'INS_REG', 'INS_DOCSER', 'OWN_DOCSER']

for index, f in enumerate(rtype):
    i = index+9
    for c in ctype:
        name = 'f' + str(i) + '_' + f + '_' + str(c)
        cmc = claimmeansclass(f, c)
        print(f, c, name)
        dt[name] = dt.progress_apply(lambda x: cmc.claimmeans(x[f], limit=10), axis = 1)

  0%|                                                                             | 1/107081 [00:00<4:12:00,  7.08it/s]

OWN_REG True f9_OWN_REG_True


  0%|                                                                             | 1/107081 [00:00<3:24:38,  8.72it/s]

OWN_REG False f9_OWN_REG_False


  0%|                                                                             | 1/107081 [00:00<4:16:07,  6.97it/s]

...

  0%|                                                                                       | 0/107081 [00:00<?, ?it/s]

OWN_DOCSER True f12_OWN_DOCSER_True


  0%|                                                                             | 1/107081 [00:00<4:11:31,  7.10it/s]

OWN_DOCSER False f12_OWN_DOCSER_False


100%|████████████████████████████████████████████████████████████████████████| 107081/107081 [00:12<00:00, 8708.44it/s]


In [23]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107081 entries, 0 to 107080
Data columns (total 77 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              107081 non-null  int64  
 1   AGR_ISN                 107081 non-null  int64  
 2   AGR_DATESIGN            107081 non-null  object 
 3   AGR_DATEBEG             107081 non-null  object 
 4   AGR_DATEEND             107081 non-null  object 
 5   AGR_SRC                 107081 non-null  object 
 6   AGR_CREATEDBY           107081 non-null  object 
 7   INS_ISN                 107081 non-null  int64  
 8   INS_NAME                107081 non-null  object 
 9   INS_SEX                 86784 non-null   object 
 10  INS_YEAR                86933 non-null   float64
 11  INS_JUR                 107081 non-null  int64  
 12  INS_REG                 107081 non-null  float64
 13  INS_DOCSER              107081 non-null  int64  
 14  OWN_ISN             

In [24]:
dt['f13_own_is_ins'] = dt.apply(lambda x: 1 if x['INS_NAME'] == x['OWN_NAME'] else 0, axis=1)

In [25]:
# после каждого шага сохраняем датасет. Крайняя фича - f13
dt.to_csv('../data/step2.csv')

In [26]:
# КБМ по договорам с юрлицами отсутствует, заполним его средними значениями КБМ по другим договорам
dt['AGR_KBM'].fillna(dt['AGR_KBM'].mean(), inplace=True)
dt.AGR_KBM.unique()

array([0.9 , 0.85, 0.8 , 1.  , 0.75, 0.95, 0.7 , 0.65, 0.6 , 0.5 , 0.55,
       0.84, 1.4 , 1.55, 2.45, 2.3 , 0.77, 0.79, 0.62, 0.83, 0.69, 1.01,
       0.82, 0.88, 0.86, 0.81, 0.68, 0.91, 1.03, 1.02, 0.93, 0.66, 0.97,
       0.98, 1.18, 0.74, 0.87, 0.72, 0.73, 0.92, 0.71, 0.67, 0.96, 0.76,
       0.94, 0.89, 0.78, 0.99, 1.07, 0.63, 0.64, 1.2 , 1.1 , 1.04, 1.08,
       1.05, 1.06, 1.19, 1.44, 1.29, 0.56, 1.11, 1.12, 1.17, 1.23, 1.15,
       1.21, 1.25, 1.28, 0.53, 0.52, 1.09, 1.13, 1.58, 0.61, 1.24, 1.3 ,
       0.59, 1.43, 0.58])

In [27]:
# выведем отдельно признак пролонгируемого договора
dt['f14_is_prolongation'] = dt['AGR_PREVISN'].apply(lambda x: 1 if x>0 else 0)

In [28]:
dt.f14_is_prolongation.unique()

array([1, 0], dtype=int64)

In [75]:
# пол, цель использования, категорию, и тип ТС закодируем в onehot, где он неизвестен (премущественно это мультидрайв) будет 0,0
one_hot = pd.get_dummies(dt[['OWN_SEX', 'INS_SEX', 'TS_TYPE', 'TS_CATEGORY', 'TS_USE_NAME']])
dt = dt.join(one_hot)

In [55]:
# после каждого шага сохраняем датасет. Крайняя фича - f14
dt.to_csv('../data/step3.csv')
pd.read_csv('../data/step3.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107081 entries, 0 to 107080
Columns: 109 entries, Unnamed: 0 to TS_CATEGORY_Категория "Tm"
dtypes: float64(40), int64(43), object(26)
memory usage: 89.0+ MB


In [56]:
# отдельными колонками выведем общее количество водителей
dt['DRV_ISN'].fillna(0, inplace=True)
dt['f15_drivers_count'] = dt['DRV_ISN'].apply(lambda x: 0 if x == 0 else len(str(x).split(',')))
dt.f15_drivers_count.unique()

array([ 0,  3,  5,  4,  1,  2,  6,  7,  8,  9, 19, 18, 10, 11, 12, 15, 13,
       16, 17], dtype=int64)

In [57]:
dt[['AGR_ISN', 'DRV_ISN', 'f15_drivers_count']]

,AGR_ISN,DRV_ISN,f15_drivers_count
0,885994559,0,0
1,886049726,"157142000,127713000,126439000",3
2,886482027,"157142000,127713000,126439000,166383000,442893564",5
3,886618143,"157142000,127713000,126439000,166383000",4
4,886683292,"442943161,157142000,127713000,126439000",4
...,...,...,...
107076,895436371,"446284314,449258526,446284317",3
107077,893007823,"446284314,449258526,446284317",3
107078,892962673,"449279084,446218114",2
107079,892715453,"446064954,449084075",2


In [58]:
def minmaxfunc(x):
    l = str(x).split(',')
    lf = [float(i) for i in l]
    return lf

minmaxfunc('1,3,4,5')

[1.0, 3.0, 4.0, 5.0]

In [59]:
# отдельными колонками выведем минимальный и максимальный возраст, стаж, и КБМ 
# (КБМ только мин, макс и так в договоре есть)
dt['DRV_KBM'].fillna(dt['AGR_KBM'].mean(), inplace=True)
dt['f23_kbm_min'] = dt['DRV_KBM'].apply(lambda x: min(minmaxfunc(x)))

In [60]:
dt['f17_drv_age_min'] = dt['DRV_YEAR'].apply(lambda x: min(minmaxfunc(x)))
dt['f18_drv_age_max'] = dt['DRV_YEAR'].apply(lambda x: max(minmaxfunc(x)))

In [61]:
# дополнительно выведем информацию о том, что собственник = страхователю и он же водит

def oneperson(s):
    if s['INS_NAME'] == s['OWN_NAME'] and s['OWN_NAME'] in str(s['DRV_NAME']).split(','):
        return 1
    else:
        return 0

In [62]:
dt['f21_oneperson'] = dt.apply(oneperson, axis=1)

In [63]:
# забыл вывести длительность договора
dt['f22_agr_days'] = dt.apply(lambda x: daysbetween(x['AGR_DATEBEG'], x['AGR_DATEEND']), axis=1)

In [64]:
# после каждого шага сохраняем датасет. Крайняя фича - f20
dt.to_csv('../data/step4.csv')

In [67]:
dt['claim'] = dt['CNT_CLAIM_PAY'].apply(lambda x: 0 if x == 0 else 1)

In [85]:
dt['INS_YEAR'].fillna(dt['INS_YEAR'].mean(), inplace=True)
dt['OWN_YEAR'].fillna(dt['OWN_YEAR'].mean(), inplace=True)
dt['f19_drv_skill_min'].fillna(dt['INS_YEAR'].mean(), inplace=True)
dt['f19_drv_skill_min'].fillna(dt['OWN_YEAR'].mean(), inplace=True)
dt['f17_drv_age_min'].fillna(dt['f17_drv_age_min'].mean(), inplace=True)
dt['f18_drv_age_max'].fillna(dt['f18_drv_age_max'].mean(), inplace=True)
dt['f19_drv_skill_min'].fillna(dt['f19_drv_skill_min'].mean(), inplace=True)
dt['f20_drv_skill_max'].fillna(dt['f20_drv_skill_max'].mean(), inplace=True)

In [86]:
print(dt.columns[50:])

Index(['COURT_EXISTS', 'COURT_AMOUNT', 'INS_ANTI_VIP', 'OWN_ANTI_VIP',
       'DRV_ANTI_VIP', 'f1_daysbeforestart', 'f2_AGR_SRC_True',
       'f2_AGR_SRC_False', 'f3_AGR_CREATEDBY_True', 'f3_AGR_CREATEDBY_False',
       'f4_TS_TYPE_True', 'f4_TS_TYPE_False', 'f5_TS_CATEGORY_True',
       'f5_TS_CATEGORY_False', 'f6_TS_MARK_NAME_True', 'f6_TS_MARK_NAME_False',
       'f7_TS_MODEL_NAME_True', 'f7_TS_MODEL_NAME_False',
       'f8_car_age_on_start', 'f9_OWN_REG_True', 'f9_OWN_REG_False',
       'f10_INS_REG_True', 'f10_INS_REG_False', 'f11_INS_DOCSER_True',
       'f11_INS_DOCSER_False', 'f12_OWN_DOCSER_True', 'f12_OWN_DOCSER_False',
       'f13_own_is_ins', 'f14_is_prolongation', 'f15_drivers_count',
       'f17_drv_age_min', 'f18_drv_age_max', 'f19_drv_skill_min',
       'f20_drv_skill_max', 'f16_kbm_min', 'f21_oneperson', 'f22_agr_days',
       'claim', 'OWN_SEX_Ж', 'OWN_SEX_М', 'INS_SEX_Ж', 'INS_SEX_М',
       'TS_TYPE_Автобусы', 'TS_TYPE_Грузовые автомобили',
       'TS_TYPE_Колесные 

In [87]:
data = dt[['INS_JUR', 'OWN_JUR', 'AGR_KBM','TS_POWER', 'f1_daysbeforestart', 'f2_AGR_SRC_True',
       'f2_AGR_SRC_False', 'f3_AGR_CREATEDBY_True', 'f3_AGR_CREATEDBY_False',
       'f4_TS_TYPE_True', 'f4_TS_TYPE_False', 'f5_TS_CATEGORY_True',
       'f5_TS_CATEGORY_False', 'f6_TS_MARK_NAME_True', 'f6_TS_MARK_NAME_False',
       'f7_TS_MODEL_NAME_True', 'f7_TS_MODEL_NAME_False',
       'f8_car_age_on_start', 'f9_OWN_REG_True', 'f9_OWN_REG_False',
       'f10_INS_REG_True', 'f10_INS_REG_False', 'f11_INS_DOCSER_True',
       'f11_INS_DOCSER_False', 'f12_OWN_DOCSER_True', 'f12_OWN_DOCSER_False',
       'f13_own_is_ins', 'f14_is_prolongation', 'f15_drivers_count',
       'f17_drv_age_min', 'f18_drv_age_max', 'f19_drv_skill_min',
       'f20_drv_skill_max', 'f16_kbm_min', 'f21_oneperson', 'f22_agr_days',
       'claim', 'OWN_SEX_Ж', 'OWN_SEX_М', 'INS_SEX_Ж', 'INS_SEX_М',
       'TS_TYPE_Автобусы', 'TS_TYPE_Грузовые автомобили',
       'TS_TYPE_Колесные тракторы и дорожные машины',
       'TS_TYPE_Легковые автомобили', 'TS_TYPE_Мотоциклы и мотороллеры',
       'TS_TYPE_Отсутствует', 'TS_TYPE_Трамваи', 'TS_TYPE_Троллейбусы',
       'TS_CATEGORY_Категория "A"', 'TS_CATEGORY_Категория "A1"',
       'TS_CATEGORY_Категория "B"', 'TS_CATEGORY_Категория "C"',
       'TS_CATEGORY_Категория "D"', 'TS_CATEGORY_Категория "E"',
       'TS_CATEGORY_Категория "Tb"', 'TS_CATEGORY_Категория "Tm"', 'TS_USE_NAME_Дорожные и специальные ТС', 'TS_USE_NAME_Инкассация',
       'TS_USE_NAME_Личная', 'TS_USE_NAME_Перевозка опасных грузов',
       'TS_USE_NAME_Прокат или краткосрочная аренда', 'TS_USE_NAME_Прочее',
       'TS_USE_NAME_Регулярные пассажирские перевозки',
       'TS_USE_NAME_Такси, Маршрутное такси', 'TS_USE_NAME_Учебная езда',
       'TS_USE_NAME_Экстренные и коммунальные службы']]

data.head(20)

,INS_JUR,OWN_JUR,AGR_KBM,TS_POWER,f1_daysbeforestart,f2_AGR_SRC_True,f2_AGR_SRC_False,f3_AGR_CREATEDBY_True,f3_AGR_CREATEDBY_False,f4_TS_TYPE_True,...,TS_USE_NAME_Дорожные и специальные ТС,TS_USE_NAME_Инкассация,TS_USE_NAME_Личная,TS_USE_NAME_Перевозка опасных грузов,TS_USE_NAME_Прокат или краткосрочная аренда,TS_USE_NAME_Прочее,TS_USE_NAME_Регулярные пассажирские перевозки,"TS_USE_NAME_Такси, Маршрутное такси",TS_USE_NAME_Учебная езда,TS_USE_NAME_Экстренные и коммунальные службы
0,1,1,0.90,180.0,2,0.346128,0.40601,0.063680,0.090337,0.060063,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0.85,156.4,3,0.346128,0.40601,0.050465,0.067729,0.060063,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0.80,235.0,5,0.346128,0.40601,0.050465,0.067729,0.060063,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0.80,235.0,5,0.346128,0.40601,0.040590,0.044280,0.060063,...,0,0,1,0,0,0,0,0,0,0
4,0,0,1.00,156.4,6,0.346128,0.40601,0.050465,0.067729,0.060063,...,0,0,1,0,0,0,0,0,0,0
5,0,0,0.75,235.0,10,0.346128,0.40601,0.047003,0.086244,0.060063,...,0,0,1,0,0,0,0,0,0,0
6,0,0,0.95,156.4,2,0.346128,0.40601,0.065512,0.121666,0.060063,...,0,0,1,0,0,0,0,0,0,0
7,0,0,0.95,156.4,2,0.346128,0.40601,0.047003,0.086244,0.060063,...,0,0,1,0,0,0,0,0,0,0
8,0,0,0.70,235.0,4,0.346128,0.40601,0.047003,0.086244,0.060063,...,0,0,1,0,0,0,0,0,0,0
9,0,0,0.90,156.4,17,0.346128,0.40601,0.046427,0.096084,0.060063,...,0,0,1,0,0,0,0,0,0,0


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107081 entries, 0 to 107080
Data columns (total 67 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   INS_JUR                                        107081 non-null  int64  
 1   OWN_JUR                                        107081 non-null  int64  
 2   AGR_KBM                                        107081 non-null  float64
 3   TS_POWER                                       106806 non-null  float64
 4   f1_daysbeforestart                             107081 non-null  int64  
 5   f2_AGR_SRC_True                                107081 non-null  float64
 6   f2_AGR_SRC_False                               107081 non-null  float64
 7   f3_AGR_CREATEDBY_True                          107081 non-null  float64
 8   f3_AGR_CREATEDBY_False                         107081 non-null  float64
 9   f4_TS_TYPE_True                      

In [89]:
data = data.dropna()

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106806 entries, 0 to 107080
Data columns (total 67 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   INS_JUR                                        106806 non-null  int64  
 1   OWN_JUR                                        106806 non-null  int64  
 2   AGR_KBM                                        106806 non-null  float64
 3   TS_POWER                                       106806 non-null  float64
 4   f1_daysbeforestart                             106806 non-null  int64  
 5   f2_AGR_SRC_True                                106806 non-null  float64
 6   f2_AGR_SRC_False                               106806 non-null  float64
 7   f3_AGR_CREATEDBY_True                          106806 non-null  float64
 8   f3_AGR_CREATEDBY_False                         106806 non-null  float64
 9   f4_TS_TYPE_True                      

In [91]:
# сохраним итоговый вариант
data.to_csv('../data/model1.csv')